# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core import ScriptRunConfig 
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'quyen-breast-cancer-hyperdrive'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quyenudacity
Azure region: eastus
Subscription id: 813e37e4-73d3-4513-a700-f9581423d59c
Resource group: happiness


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "quyen-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 5)

Found existing cluster, use it.


In [4]:
from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split

key = 'breast-cancer-dataset'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("Dataset found!")

else:
    print("Register dataset")

Dataset found!


In [5]:
df = dataset.to_pandas_dataframe()
train, test = train_test_split(df, shuffle=True)
train.to_csv('train.csv',index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore,'train.csv')])

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
Uploading an estimated of 1 files
Uploading ./train.csv
Uploaded ./train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [9]:
from azureml.core import Environment, ScriptRunConfig
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
    '--n_estimator': choice(100, 200, 500, 800, 1000),
    '--min_samples_split': choice(2, 5, 10),
    '--max_features': choice('auto', 'sqrt', 'log2'),
    '--bootstrap': choice(1, 0)
})

if "training" not in os.listdir():
    os.mkdir("./training")

# get registered environment
env = Environment.from_conda_specification(name="sklearn-env",file_path="./dependencies.yaml")

# set up script run configuration
estimator = ScriptRunConfig(
    source_directory='./',
    script='train.py',
    compute_target=compute_target,
    environment=env
)

hyperdrive_run_config = HyperDriveConfig(run_config = estimator,
                                     hyperparameter_sampling = param_sampling, 
                                     policy = early_termination_policy,
                                     primary_metric_name = 'Accuracy',
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 10,
                                     max_concurrent_runs = 5)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_e47f9e4d-5dd8-4efb-8b14-88435e76ac2f
Web View: https://ml.azure.com/runs/HD_e47f9e4d-5dd8-4efb-8b14-88435e76ac2f?wsid=/subscriptions/813e37e4-73d3-4513-a700-f9581423d59c/resourcegroups/happiness/workspaces/quyenudacity&tid=f01e930a-b52e-42b1-b70f-a8882b5d043b

Streaming azureml-logs/hyperdrive.txt

[2024-09-13T12:51:01.000655][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space
[2024-09-13T12:51:01.7738130Z][SCHEDULER][INFO]Scheduling job, id='HD_e47f9e4d-5dd8-4efb-8b14-88435e76ac2f_0' 
[2024-09-13T12:51:01.8926106Z][SCHEDULER][INFO]Scheduling job, id='HD_e47f9e4d-5dd8-4efb-8b14-88435e76ac2f_1' 
[2024-09-13T12:51:01.9610959Z][SCHEDULER][INFO]Scheduling job, id='HD_e47f9e4d-5dd8-4efb-8b14-88435e76ac2f_2' 
[2024-09-13T12:51:02.1129389Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_e47f9e4d-5dd8-4efb-8b14-88435e76ac2f_0' 
[2024-09-13T12:51:02.1853456Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_e47f9e4d-5dd8-4efb-8b14-88435e76ac2f_1' 
[2024-

{'runId': 'HD_e47f9e4d-5dd8-4efb-8b14-88435e76ac2f',
 'target': 'quyen-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-09-13T12:50:59.272521Z',
 'endTimeUtc': '2024-09-13T12:54:01.820245Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '6e3f8a4b-44fa-4068-9fbd-b91e56767c57',
  'user_agent': 'python/3.9.19 (Linux-5.15.0-1064-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.56.0',
  'space_size': '90',
  'best_child_run_id': 'HD_e47f9e4d-5dd8-4efb-8b14-88435e76ac2f_1',
  'score': '0.9707602339181286',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_e47f9e4d-5dd8-4efb-8b14-88435e76ac2f_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClie

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details())

{'runId': 'HD_e47f9e4d-5dd8-4efb-8b14-88435e76ac2f_1', 'target': 'quyen-cluster', 'status': 'Completed', 'startTimeUtc': '2024-09-13T12:51:50.384613Z', 'endTimeUtc': '2024-09-13T12:52:06.928325Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', '_azureml.ClusterName': 'quyen-cluster', 'ContentSnapshotId': '6e3f8a4b-44fa-4068-9fbd-b91e56767c57', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--bootstrap', '1', '--max_features', 'sqrt', '--min_samples_split', '2', '--n_estimator', '1000'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'quyen-cluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTypes': [], 'priority': 

In [15]:
#TODO: Save the best model
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

No of Estimators: 1000
Min No of Samples to Split: 2
No of Features Considered: sqrt
Bootstrap: True
Accuracy 0.9707602339181286


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [17]:
model = best_run.register_model(model_name='hyperdrive-model', model_path='outputs/model.pkl')

In [18]:
from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description='Predict tumor diagnosis - Malignant(1) or Benign(0)',
    auth_enabled=True
)

In [23]:
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.automl.core.shared import constants

model = Model(ws, 'hyperdrive-model')


myenv = best_run.get_environment()
entry_script = 'predict.py'
#best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)
#best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
inference_config = InferenceConfig(entry_script=entry_script, environment=myenv)

service = Model.deploy(workspace=ws, 
                       name='hyperdrive-breastcancer', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aci_config)

service.wait_for_deployment(show_output=True)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-09-13 13:33:18+00:00 Registering the environment.
2024-09-13 13:33:18+00:00 Use the existing image.
2024-09-13 13:33:19+00:00 Generating deployment configuration.
2024-09-13 13:33:21+00:00 Submitting deployment to compute.
2024-09-13 13:33:29+00:00 Checking the status of deployment hyperdrive-breastcancer..
2024-09-13 13:35:24+00:00 Checking the status of inference endpoint hyperdrive-breastcancer.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [24]:
!python endpoint.py

['B', 'M']


In [25]:
print(service.get_logs())

2024-09-13T13:35:01,681538000+00:00 - rsyslog/run 
2024-09-13T13:35:01,698093900+00:00 - gunicorn/run 
2024-09-13T13:35:01,700395100+00:00 | gunicorn/run | 
2024-09-13T13:35:01,706219200+00:00 | gunicorn/run | ###############################################
2024-09-13T13:35:01,707801000+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-09-13T13:35:01,715277500+00:00 | gunicorn/run | ###############################################
2024-09-13T13:35:01,717377100+00:00 | gunicorn/run | 
2024-09-13T13:35:01,717631500+00:00 - nginx/run 
2024-09-13T13:35:01,724518000+00:00 | gunicorn/run | 
2024-09-13T13:35:01,735083900+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20240418.v1
2024-09-13T13:35:01,737022300+00:00 | gunicorn/run | 
2024-09-13T13:35:01,742513900+00:00 | gunicorn/run | 
2024-09-13T13:35:01,749427300+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml_cae9639363fd641d6f309ea039aa84b1/bin:/opt/mi

In [ ]:
service.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

